In [1]:
from keras.utils import to_categorical
from keras.optimizers import Adam, Adadelta, SGD
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten, Conv1D, GlobalAveragePooling1D, Convolution1D
from keras.layers.convolutional import MaxPooling2D, MaxPooling1D
from keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import class_weight
from scipy import signal
from glob import glob
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

c:\users\sebastiaan\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# constants:
SUBSAMPLE = 10  # scales the data by this factor
COLUMNS = ['HandStart', 'FirstDigitTouch',
        'BothStartLoadPhase', 'LiftOff',
        'Replace', 'BothReleased']
TRAIN_SUBJECTS = range(1, 9)
TEST_SUBJECTS = range(9, 13)
N_LABELS = 6

# Below are the paths to the data. Please pay attention to the % and * signs, these are needed in the loop.
TRAIN_DATA_PATH = 'C:/Users/Sebastiaan/Desktop/Programming/MachineLearning/Datasets/EEG/train/subj%d_series*_data.csv'
# TRAIN_DATA_PATH = 'C:/Users/bas/Documents/MachineLearning/train/subj%d_series*_data.csv'

EPOCHS = 100
WINDOW_SIZE = 2000
SPLIT_SIZE = 50
BATCH_SIZE = 10

In [3]:
def prepare_training_data(data_path):
    data = pd.read_csv(data_path)
    event_path = data_path.replace('_data', '_events')
    labels = pd.read_csv(event_path)
    clean_data = data.drop(['id'], axis = 1)
    labels = labels.drop(['id'], axis = 1)
    return clean_data, labels

In [4]:
def preprocess_data(x_raw, WINDOW_SIZE, subsample):
    x_raw, scaler = scaler_transform(x_raw[::subsample], None)
    x_raw = image_mappping(x_raw, WINDOW_SIZE)
    return x_raw, scaler

In [5]:
def scaler_transform(data, scaler):
    if scaler == None:
        scaler = Normalizer()
        return scaler.fit_transform(data), scaler
    else:
        return scaler.transform(data)   

In [6]:
def read_training_data(train_data_paths):
    labels_raw = []
    features_raw = []
    for data_path in train_data_paths:
        data, labels = prepare_training_data(data_path)
        features_raw.append(data)
        labels_raw.append(labels)
    features_raw = pd.concat(features_raw)
    labels_raw = pd.concat(labels_raw)
    x_train = np.asarray(features_raw.astype(float))
    y_train = np.asarray(labels_raw.astype(float))
    return x_train, y_train

In [7]:
def metric_auc_score(predictions, y_test, with_plot):
    scores = []
    legend_text = []
    for i in range(N_LABELS):
        fpr, tpr, _  = roc_curve(y_test[:,i], predictions[:,i], 1)
        scores.append(roc_auc_score(y_test[:,i], predictions[:,i]))
        legend_text.append(COLUMNS[i]+' (area = %.3f)' % (scores[i]))
        if with_plot == True:
            plt.plot(fpr, tpr)
    if with_plot == True:
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves')
        plt.legend(legend_text)
        plt.show()
    return scores

In [8]:
def multiple_metric_auc_score(prediction_total, test_data_total, with_plot):
    legend_text = []
    counter = 0
    for i in range(len(prediction_total)):
        fpr, tpr, _  = roc_curve(test_data_total[i], prediction_total[i], pos_label=1)
        score = roc_auc_score(test_data_total[i],prediction_total[i])
        legend_text.append(COLUMNS[i]+' (area = %.3f)' % (score))
        print(COLUMNS[i]+' AUC score = %.3f' % (score))
        plt.plot(fpr, tpr)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves')
        plt.legend(legend_text)
    plt.show()

In [9]:
def single_metric_auc_score(predictions, y_test, with_plot, i):
    fpr, tpr, _  = roc_curve(y_test, predictions, 1)
    score = roc_auc_score(y_test, predictions)
    print(COLUMNS[i]+' AUC score = %.3f' % (score))
    if with_plot == True:
        plt.plot(fpr, tpr)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves')
        plt.show()
    return score

In [10]:
def all_auc_scores(prediction_total, test_data_total, subjects, with_plot):
    scores = []
    for subject in subjects:
        score = metric_auc_score(prediction_total[subject-1],
                                 test_data_total[subject-1], with_plot)
        scores.append(score)
        print('Mean AUC Score of Subject %d: %.3f' % \
              (subject, np.mean(score)))
    return scores

In [11]:
def image_mappping(x_train, WINDOW_SIZE):
    result = []
    empty_matrix = np.zeros(np.shape(x_train[0:WINDOW_SIZE]))
    for i in range(len(x_train)):
        if i-WINDOW_SIZE < 0:
            result.append(empty_matrix)
        else:
            result.append(x_train[i-WINDOW_SIZE:i])
    return result

In [12]:
def resample_and_shuffle(labels):
    when_task = np.where(labels == 1)
    when_no_task = np.where(labels == 0)
    when_no_task = when_no_task[0][0:len(when_task[0])]
    indices = np.concatenate([when_task[0], when_no_task])
    np.random.shuffle(indices)
    return indices

In [13]:
def balance_set(indices, x_train, y_train):
    balance_x = []
    balance_y = []
    for index in indices:
        balance_x.append(x_train[index])
        balance_y.append(y_train[index])
    return np.array(balance_x), np.array(balance_y)

In [14]:
def remove_imbalance(x_train, y_train):
    indices = resample_and_shuffle(y_train)
    balanced_x_train, balanced_y_train = balance_set(indices, x_train, y_train)
    return balanced_x_train, balanced_y_train

In [15]:
def predict_on_sub(x_test, model, split_size, batch_size):
    split_size = int(len(x_test) / split_size)
    sub_x_test = x_test[1000::batch_size]
    batch = []
    predictions = np.array([])
    for i in range(len(sub_x_test)):
        batch.append(sub_x_test[i])
        if i+1 == len(sub_x_test):
            return np.concatenate([predictions, model.predict(np.array(batch))[:,1]])
        elif (i+1) % split_size == 0:
            predictions = np.concatenate([predictions, model.predict(np.array(batch))[:,1]])
            batch = []

In [16]:
def init_cnn(window):
    model = Sequential()
    model.add(Conv1D(nb_filter=16, filter_length=3, activation='relu', input_shape=(window, 32)))
    model.add(MaxPooling1D(3,2))
    model.add(Conv1D(nb_filter=32, filter_length=3, activation='relu'))
    model.add(Conv1D(nb_filter=64, filter_length=3, activation='relu'))
    model.add(MaxPooling1D(3,2))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
#     model.summary()
    
    optimizer = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
#     optimizer = Adam(lr=1e-6)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
prediction_total = []
test_data_total = []

models = []
for i in range(N_LABELS):
    models.append(init_cnn(WINDOW_SIZE))

for subject in TRAIN_SUBJECTS:
    test_features_raw = []
    train_data_paths = glob(TRAIN_DATA_PATH % (subject))

    
    x_raw, y_raw = read_training_data(train_data_paths)
    
    x_raw, _ = preprocess_data(x_raw, WINDOW_SIZE, SUBSAMPLE)
#     y_train = y_raw[::SUBSAMPLE]
    x_train, _, y_train, _ = train_test_split(x_raw, y_raw[::SUBSAMPLE], test_size=0.5)

    
    for i in range(N_LABELS):
        model = models[i]
        
        balanced_x_train, balanced_y_train = remove_imbalance(x_train, y_train[:,i])
        
        train_labels = to_categorical(balanced_y_train, num_classes = None)
                
        print('Train subject %d, class %s' % (subject, COLUMNS[i]))        
        model.fit(balanced_x_train, train_labels, verbose=1, validation_split=0.2, epochs=EPOCHS)
    
    
    for task_name, model in zip(COLUMNS, models):
        model.save('general_model_'+task_name+'_subject_%d.h5' % (subject))  # creates a HDF5 file 'my_model.h5'
#         predictions = predict_on_sub(x_test, model, SPLIT_SIZE, BATCH_SIZE)
        
#         test_data_total.append(y_test[:,i][1000::BATCH_SIZE])
#         prediction_total.append(predictions)
        
#     multiple_metric_auc_score(prediction_total, test_data_total, True)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train subject 1, class HandStart
Train on 3075 samples, validate on 769 samples
Epoch 1/100
3075/3075 [==============================] - 10s 3ms/step - loss: 0.6956 - acc: 0.4966 - val_loss: 0.6927 - val_acc: 0.5189
Epoch 2/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.6937 - acc: 0.5037 - val_loss: 0.6916 - val_acc: 0.5033
Epoch 3/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.6920 - acc: 0.5154 - val_loss: 0.6899 - val_acc: 0.5371
Epoch 4/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.6909 - acc: 0.5207 - val_loss: 0.6885 - val_acc: 0.5280
Epoch 5/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.6892 - acc: 0.5499 - val_loss: 0.6865 - val_acc: 0.5878
Epoch 6/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.6858 - acc: 0.5662 - val_loss: 0.6851 - val_acc: 0.5488
Epoch 7/100
3075/3075 [===

3075/3075 [==============================] - 5s 2ms/step - loss: 0.4118 - acc: 0.8237 - val_loss: 0.3898 - val_acc: 0.8427
Epoch 59/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.4106 - acc: 0.8267 - val_loss: 0.3992 - val_acc: 0.8244
Epoch 60/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.4170 - acc: 0.8263 - val_loss: 0.3868 - val_acc: 0.8401
Epoch 61/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.4109 - acc: 0.8250 - val_loss: 0.4057 - val_acc: 0.8218
Epoch 62/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.4091 - acc: 0.8237 - val_loss: 0.3833 - val_acc: 0.8414
Epoch 63/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.4002 - acc: 0.8280 - val_loss: 0.3795 - val_acc: 0.8440
Epoch 64/100
3075/3075 [==============================] - 5s 2ms/step - loss: 0.4008 - acc: 0.8286 - val_loss: 0.3768 - val_acc: 0.8518
Epoch 65/100
3075/3075 [==============================] - 5s 

3104/3104 [==============================] - 5s 2ms/step - loss: 0.6837 - acc: 0.5915 - val_loss: 0.6792 - val_acc: 0.5941
Epoch 18/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.6790 - acc: 0.6128 - val_loss: 0.6764 - val_acc: 0.6572
Epoch 19/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.6782 - acc: 0.6266 - val_loss: 0.6737 - val_acc: 0.6985
Epoch 20/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.6771 - acc: 0.6134 - val_loss: 0.6723 - val_acc: 0.6675
Epoch 21/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.6743 - acc: 0.6263 - val_loss: 0.6692 - val_acc: 0.6804
Epoch 22/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.6712 - acc: 0.6398 - val_loss: 0.6664 - val_acc: 0.6817
Epoch 23/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.6697 - acc: 0.6569 - val_loss: 0.6638 - val_acc: 0.6778
Epoch 24/100
3104/3104 [==============================] - 5s 

3104/3104 [==============================] - 5s 2ms/step - loss: 0.2924 - acc: 0.8843 - val_loss: 0.3196 - val_acc: 0.8686
Epoch 78/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.2835 - acc: 0.8805 - val_loss: 0.3592 - val_acc: 0.8608
Epoch 79/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.2862 - acc: 0.8879 - val_loss: 0.3344 - val_acc: 0.8698
Epoch 80/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.2738 - acc: 0.8918 - val_loss: 0.3211 - val_acc: 0.8750
Epoch 81/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.2722 - acc: 0.8898 - val_loss: 0.3310 - val_acc: 0.8737
Epoch 82/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.2711 - acc: 0.8918 - val_loss: 0.3528 - val_acc: 0.8570
Epoch 83/100
3104/3104 [==============================] - 5s 2ms/step - loss: 0.2726 - acc: 0.8898 - val_loss: 0.3315 - val_acc: 0.8711
Epoch 84/100
3104/3104 [==============================] - 5s 

3097/3097 [==============================] - 5s 2ms/step - loss: 0.5410 - acc: 0.7359 - val_loss: 0.5472 - val_acc: 0.7652
Epoch 37/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.5221 - acc: 0.7614 - val_loss: 0.5485 - val_acc: 0.7394
Epoch 38/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.5150 - acc: 0.7614 - val_loss: 0.5173 - val_acc: 0.7871
Epoch 39/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.5041 - acc: 0.7707 - val_loss: 0.5052 - val_acc: 0.7871
Epoch 40/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.4877 - acc: 0.7791 - val_loss: 0.4911 - val_acc: 0.7961
Epoch 41/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.4745 - acc: 0.7814 - val_loss: 0.4802 - val_acc: 0.7974
Epoch 42/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.4658 - acc: 0.7908 - val_loss: 0.4903 - val_acc: 0.7897
Epoch 43/100
3097/3097 [==============================] - 5s 

3097/3097 [==============================] - 5s 2ms/step - loss: 0.2124 - acc: 0.9222 - val_loss: 0.2431 - val_acc: 0.8981
Epoch 97/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.2069 - acc: 0.9219 - val_loss: 0.2475 - val_acc: 0.8981
Epoch 98/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.2052 - acc: 0.9299 - val_loss: 0.2286 - val_acc: 0.9123
Epoch 99/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.2064 - acc: 0.9206 - val_loss: 0.2048 - val_acc: 0.9277
Epoch 100/100
3097/3097 [==============================] - 5s 2ms/step - loss: 0.1992 - acc: 0.9238 - val_loss: 0.2703 - val_acc: 0.8955
Train subject 1, class LiftOff
Train on 3112 samples, validate on 778 samples
Epoch 1/100
3112/3112 [==============================] - 6s 2ms/step - loss: 0.6966 - acc: 0.4978 - val_loss: 0.6928 - val_acc: 0.5000
Epoch 2/100
3112/3112 [==============================] - 5s 1ms/step - loss: 0.6942 - acc: 0.5122 - val_loss: 0.6906 - 

In [ ]:
# from keras.models import load_model
# for task_name, model in zip(COLUMNS, models):
#     model.save('general_model_'+task_name+'.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
# model = load_model('general_model_handstart.h5')

In [ ]:


for subject in TEST_SUBJECTS:
    prediction_total = []
    test_data_total = []
    test_features_raw = []
    train_data_paths = glob(TRAIN_DATA_PATH % (subject))

    
    x_raw, y_raw = read_training_data(train_data_paths)
    
    x_raw, _ = preprocess_data(x_raw, WINDOW_SIZE, SUBSAMPLE)

    _, x_test, _, y_test = train_test_split(x_raw, y_raw[::SUBSAMPLE], test_size=0.5)

    
    for i in range(N_LABELS):

        test_labels = to_categorical(y_test[:,i], num_classes = None)
                
        print('Test subject %d, class %s' % (subject, COLUMNS[i]))                
        predictions = predict_on_sub(x_test, models[i], SPLIT_SIZE, BATCH_SIZE)
        
        test_data_total.append(y_test[:,i][1000::BATCH_SIZE])
        prediction_total.append(predictions)
        
    multiple_metric_auc_score(prediction_total, test_data_total, True)